In [ ]:
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from scipy import stats 
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import seaborn as sns
import matplotlib.pyplot as plt
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [ ]:
enefit = pd.read_csv("./predict-energy-behavior-of-prosumers/train.csv")
enefit["datetime"] = pd.to_datetime(enefit["datetime"])
enefit["date"] =  enefit["datetime"].dt.date

consumption = enefit[enefit["is_consumption"]==1]
consumption_mean = consumption.target.mean()
consumption_daily = consumption.groupby(['date'])['target'].mean().reset_index()

production = enefit[enefit["is_consumption"]==0]
production_mean = production.target.mean()
production_daily = production.groupby(['date'])['target'].mean().reset_index()

In [ ]:
gas = pd.read_csv("./predict-energy-behavior-of-prosumers/gas_prices.csv")
# gas.drop(["origin_date"],inplace=True,axis=1)
gas["forecast_date"] = pd.to_datetime(gas["forecast_date"])

daily_gas = gas.groupby(pd.Grouper(key="forecast_date", freq='D')).mean() 
meanGasLow = gas.lowest_price_per_mwh.mean()
meanGasHigh = gas.highest_price_per_mwh.mean()

In [ ]:
electricity = pd.read_csv("./predict-energy-behavior-of-prosumers/electricity_prices.csv")
# electricity.drop(["origin_date"],inplace=True,axis=1)
electricity["forecast_date"] = pd.to_datetime(electricity["forecast_date"])

daily_elec = electricity.groupby(pd.Grouper(key="forecast_date", freq='D')).mean() 
meanElec = electricity.euros_per_mwh.mean()

In [ ]:
historical_weather = pd.read_csv("./predict-energy-behavior-of-prosumers/historical_weather.csv")
historical_weather["datetime"] = pd.to_datetime(historical_weather["datetime"])

historical_weather_daily = historical_weather.groupby(pd.Grouper(key="datetime", freq='D')).mean() 
historical_weather_meanTemp = historical_weather.temperature.mean()
historical_weather_meanSolar = historical_weather.direct_solar_radiation.mean()
historical_weather_meanShort = historical_weather.shortwave_radiation.mean()
historical_weather_meanRain = historical_weather.rain.mean()
historical_weather_meanSnow = historical_weather.snowfall.mean()
historical_weather_meanWindspeed = historical_weather.windspeed_10m.mean()
historical_weather_meanSurfacePressure = historical_weather.surface_pressure.mean()

In [ ]:
forecast_weather = pd.read_csv("./predict-energy-behavior-of-prosumers/forecast_weather.csv")
forecast_weather["origin_datetime"] = pd.to_datetime(forecast_weather["origin_datetime"])

forecast_weather_daily = forecast_weather.groupby(pd.Grouper(key="origin_datetime", freq='D')).mean() 
forecast_weather_meanTemp = forecast_weather.temperature.mean()
forecast_weather_meanSolar = forecast_weather.direct_solar_radiation.mean()
forecast_weather_meanSnow = forecast_weather.snowfall.mean()

# Visualization
- Original file의 raw data 시각화하여 전반적인 데이터 파악

## Consumption / Production
- kaggle에서 예측하려고 하는 것은 'train.csv'의 에너지 사용량/생산량을 의미하는 'target' column
- 'train_csv'의 'is_consumption' column을 통해 에너지의 생산/소비 구분
- 실제 kaggle 에서는 hourly 예측이 목표이지만 먼저 daily로 data 탐색
- 'train.csv'의 'target' column을 시각화

In [ ]:
fig = px.area(consumption, x=consumption.datetime, y="target", title='Hourly Consumption Analysis')
fig.update_traces(line_color='#FA163F')
fig.update_layout(xaxis_title="Datetime", yaxis_title="Consumption")
fig.show()

In [ ]:
fig = px.area(consumption_daily, x=consumption_daily.date, y="target", title='Daily Consumption Analysis')
fig.add_hline(y=consumption_mean, line_dash="dot", annotation_text="Average Consumption", annotation_position="bottom right")
fig.update_traces(line_color='#FA163F')
fig.update_layout(xaxis_title="Date", yaxis_title="Consumption")
fig.show()

In [ ]:
fig = px.area(production,x=production.datetime,y="target", title='Hourly Production Analysis')
fig.update_traces(line_color='#427D9D')
fig.update_layout(xaxis_title="Date", yaxis_title="Production")
fig.show()

In [ ]:
fig = px.area(production_daily,x=production_daily.date,y="target", title='Daily Production Analysis')
fig.add_hline(y=production_mean, line_dash="dot", annotation_text="Average Daily Production", annotation_position="bottom right")
fig.update_traces(line_color='#427D9D')
fig.update_layout(xaxis_title="Date", yaxis_title="Production")
fig.show()

In [ ]:
# 에너지 순소비량 = 소비량 - 생산량
fig = px.area(consumption_daily["target"]-production_daily["target"], x=production_daily.index,y="target", title='Daily Net Consumption (Consumption-Production) Analysis')
fig.add_hline(y=(consumption_daily["target"]-production_daily["target"]).mean(), line_dash="dot", annotation_text="Average Daily Net Consumption", annotation_position="bottom right")
fig.update_traces(line_color='#EC8F5E')
fig.update_layout(xaxis_title="Date", yaxis_title="Net Consumption")
fig.show()

## Gas
- 가스 가격은 실제 생산과 거래가 이루어지는 전날의 가격으로 구매하거나 판매
- origin_date: 거래하기 전 날, 즉 가격이 형성되는 날짜
- forecast_date: 실제 거래가 이루어지는 날

In [ ]:
fig = px.area(daily_gas, x=daily_gas.index, y=["lowest_price_per_mwh","highest_price_per_mwh"], title='Daily Price/MWh Analysis', color_discrete_map={"lowest_price_per_mwh": "#EFB74F", "highest_price_per_mwh": "#247881"})
fig.add_hline(y=meanGasLow, line_dash="dot", annotation_text="Average Low Price/MWh", annotation_position="top right")
fig.add_hline(y=meanGasHigh, line_dash="dot", annotation_text="Average High Price/MWh", annotation_position="bottom right")
fig.update_layout(xaxis_title="Date", yaxis_title="Euros/MWh", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## Electricity
- 가스의 거래 방식과 같은 구조
- 'origin_date'와 'forecast_date'도 같은 개념

In [ ]:
fig = px.area(daily_elec, x=daily_elec.index, y=["euros_per_mwh"], title='Daily Price/MWh Analysis', color_discrete_map={"euros_per_mwh": "#EFB74F"})
fig.add_hline(y=meanElec, line_dash="dot", annotation_text="Average Price/MWh", annotation_position="top right")
fig.update_layout(xaxis_title="Date", yaxis_title="Euros/MWh", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## Historical Weather
- 날씨 관측 데이터
- 미래의 날씨를 예보하는 것이 아닌 실제 기록된 데이터 

In [ ]:
fig = px.area(historical_weather_daily, x=historical_weather_daily.index, y="temperature", title='Daily Historical Temperature Analysis')
fig.add_hline(y=historical_weather_meanTemp, line_dash="dot", annotation_text="Average Temperature", annotation_position="bottom right")
fig.update_traces(line_color='#C59279')
fig.update_layout(xaxis_title="Date", yaxis_title="Temperature", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
fig = px.area(historical_weather_daily, x=historical_weather_daily.index, y=["direct_solar_radiation", "shortwave_radiation"], title='Daily Radiation Analysis', color_discrete_map={"shortwave_radiation": "#6499E9", "direct_solar_radiation": "#C70A80"})
fig.add_hline(y=historical_weather_meanSolar, line_dash="dot", annotation_text="Average Solar Radiation", annotation_position="bottom right")
fig.add_hline(y=historical_weather_meanShort, line_dash="dot", annotation_text="Average Shortwave Radiation", annotation_position="top right")
fig.update_layout(xaxis_title="Date", yaxis_title="Radiation Level",legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
fig = px.area(historical_weather_daily, x=historical_weather_daily.index, y=["windspeed_10m"], title='Daily Windspeed Analysis', color_discrete_map={"windspeed_10m": "#69C98D"}, line_shape='spline')
fig.add_hline(y=historical_weather_meanWindspeed, line_dash="dot", annotation_text="Average Windspeed", annotation_position="bottom right")
fig.update_layout(xaxis_title="Date", yaxis_title="Windspeed (in m/s)", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
fig = px.area(historical_weather_daily, x=historical_weather_daily.index, y=["surface_pressure"], title='Daily Surface Pressure Analysis', color_discrete_map={"surface_pressure": "#D61640"}, line_shape='spline')
fig.add_hline(y=historical_weather_meanSurfacePressure, line_dash="dot", annotation_text="Average Surface Pressure", annotation_position="bottom right")
fig.update_layout(xaxis_title="Date", yaxis_title="Pressure (in Hectopascals)",yaxis_range=[900,1100], legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## Forecast Weather
- 전날 기록되는 날씨 예보 데이터
- origin_datetime: 예보 데이터가 만들어지는 기준 시각
- forecast_datetime: 위 기준 시각에서 만들어지는 예보 데이터, 48시간 이후까지 예보 

In [ ]:
fig = px.area(forecast_weather_daily, x=forecast_weather_daily.index, y="temperature", title='Daily Forecast Temperature Analysis')
fig.add_hline(y=forecast_weather_meanTemp, line_dash="dot",annotation_text="Average Temperature", annotation_position="bottom right")
fig.update_traces(line_color='#C59279')
fig.update_layout(xaxis_title="Date", yaxis_title="Temperature", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

# Preprocessing
- Kaggle에서는 'data_block_id'를 모든 file에 동일하게 부여
- 데이터 병합에 사용하기에 가장 적합

In [ ]:
# Should merge on 'data_block_id'
print(enefit[['datetime','data_block_id']].head(3))
print()
print(gas[['origin_date','forecast_date','data_block_id']].head(3))
print()
print(electricity[['origin_date','forecast_date','data_block_id']].head(3))
print()
print(historical_weather[['datetime','data_block_id']].head(3))
print()
print(forecast_weather[['origin_datetime','forecast_datetime','data_block_id']].head(3))


In [ ]:
enefit_daily = enefit.groupby([c for c in enefit.columns if c not in ['target', 'datetime', 'row_id', 'prediction_unit_id','date']])['target'].mean().reset_index()

enefit_daily['target'] = stats.zscore(enefit_daily['target']) 

print(enefit_daily.shape)
print(enefit_daily['data_block_id'].nunique())
enefit_daily.head()

In [ ]:
gas = pd.read_csv('./predict-energy-behavior-of-prosumers/gas_prices.csv')

# gas_hourly = gas.copy()
# gas_hourly["forecast_date"] = pd.to_datetime(gas_hourly["forecast_date"])
# gas_hourly["origin_date"] = pd.to_datetime(gas_hourly["origin_date"])
# gas_hourly = gas_hourly.set_index("forecast_date").resample("H").ffill().reset_index().rename({"forecast_date": "forecast_datetime"}, axis=1)

gas = gas.drop(['forecast_date', 'origin_date'], axis=1)

gas[['lowest_price_per_mwh', 'highest_price_per_mwh']] = stats.zscore(gas[['lowest_price_per_mwh', 'highest_price_per_mwh']]) 

gas.head()

In [ ]:
electricity = pd.read_csv("./predict-energy-behavior-of-prosumers/electricity_prices.csv")
electricity_daily = electricity.drop(['forecast_date', 'origin_date'], axis=1)
electricity_daily = electricity_daily.groupby('data_block_id').mean().reset_index()

electricity_daily['euros_per_mwh'] = stats.zscore(electricity_daily['euros_per_mwh']) 

electricity_daily.head()


In [ ]:
forecast_weather = pd.read_csv("./predict-energy-behavior-of-prosumers/forecast_weather.csv")
county_lat_lon = pd.read_csv('./predict-energy-behavior-of-prosumers/county_lon_lats.csv')

# Extracting the necessary columns (longitude and latitude) from both dataframes
weather_coords = forecast_weather[['longitude', 'latitude']].values
county_coords = county_lat_lon[['longitude', 'latitude']].values

# Creating a KDTree for efficient nearest neighbor search
kd_tree = cKDTree(county_coords)

# Finding the nearest neighbor for each point in weather_coords
# The query method returns distances and indices of the nearest neighbors
distances, indices = kd_tree.query(weather_coords)

# Using the indices to map the 'county' values from _county_lon_lats_df to forecast_weather_df
forecast_weather['county'] = county_lat_lon.loc[indices, 'county'].values

# Now, performing the requested data manipulations
# Removing specified columns
forecast_weather = forecast_weather.drop(['latitude', 'longitude', 'origin_datetime', 'forecast_datetime'], axis=1)

# Filtering out rows with 'hours_ahead' values from 1 to 24
forecast_weather = forecast_weather[~forecast_weather['hours_ahead'].between(1, 24)]

# Dropping the 'hours_ahead' column
forecast_weather = forecast_weather.drop('hours_ahead', axis=1)

# Grouping by all columns (except 'data_block_id') and calculating mean based on 'data_block_id'
forecast_weather = forecast_weather.groupby(['data_block_id', 'county']).mean().reset_index()

# Standardize of continuous features
forecast_weather_columns = forecast_weather.columns.difference(['data_block_id','county'])
forecast_weather[forecast_weather_columns] = stats.zscore(forecast_weather[forecast_weather_columns]) 

forecast_weather.head()


In [ ]:
historical_weather = pd.read_csv("./predict-energy-behavior-of-prosumers/historical_weather.csv")

# Extracting the necessary columns (longitude and latitude) from both dataframes
weather_coords = historical_weather[['longitude', 'latitude']].values
county_coords = county_lat_lon[['longitude', 'latitude']].values

# Creating a KDTree for efficient nearest neighbor search
kd_tree = cKDTree(county_coords)

# Finding the nearest neighbor for each point in weather_coords
# The query method returns distances and indices of the nearest neighbors
distances, indices = kd_tree.query(weather_coords)

# Using the indices to map the 'county' values from county_lon_lats_df to forecast_weather_df
historical_weather['county'] = county_lat_lon.loc[indices, 'county'].values

# Now, performing the requested data manipulations
# Removing specified columns
historical_weather = historical_weather.drop(['latitude', 'longitude', 'datetime', ], axis=1)

# Grouping by all columns (except 'data_block_id') and calculating mean based on 'data_block_id'
historical_weather = historical_weather.groupby(['data_block_id', 'county']).mean().reset_index()

# Standardize of continuous features
historical_weather_columns = historical_weather.columns.difference(['data_block_id','county'])
historical_weather[historical_weather_columns] = stats.zscore(historical_weather[historical_weather_columns]) 

historical_weather.head()


In [ ]:
client = pd.read_csv('./predict-energy-behavior-of-prosumers/client.csv')

# Standardize of continuous features
client[['eic_count', 'installed_capacity']] = stats.zscore(client[['eic_count', 'installed_capacity']]) 

client.head()

In [ ]:
# 날짜를 의미하는 'data_block_id'에 사용자를 의미하는 'prdiction_unit_id'를 기준으로 하는 dataframe 생성
# 'country', 'is_business', 'product_type'은 사용자의 속성을 의미

df = pd.merge(enefit_daily, gas, on='data_block_id', how='inner')
df = pd.merge(df, electricity_daily, on='data_block_id', how='inner')
df = pd.merge(df, forecast_weather, on=['data_block_id', 'county'], how='inner')
df = pd.merge(df, historical_weather, on=['data_block_id', 'county'], how='inner')
df = pd.merge(df, client, on=['data_block_id', 'product_type', 'county','is_business'], how='inner').drop('date', axis=1)
# df = df.drop('data_block_id', axis=1)

df.columns


# EDA

In [ ]:
# 사용자 속성별로 데이터 차이 구분

# Creating plots for 'target' values changed by 'data_block_id', 
# with separate plots for different values of 'is_business', 'product_type', and 'county'.

# Plot for 'target' values by 'data_block_id' for different 'is_business' values
g = sns.FacetGrid(df, col="is_business", height=4, aspect=1.5)
g.map(sns.lineplot, "data_block_id", "target")
g.set_titles("is_business = {col_name}")
g.set_axis_labels("Data Block ID", "Target Value")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Target Values by Data Block ID for Different is_business Values")
plt.show()

# Plot for 'target' values by 'data_block_id' for different 'product_type' values
g = sns.FacetGrid(df, col="product_type", col_wrap=4, height=4, aspect=1)
g.map(sns.lineplot, "data_block_id", "target")
g.set_titles("product_type = {col_name}")
g.set_axis_labels("Data Block ID", "Target Value")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Target Values by Data Block ID for Different Product Types")
plt.show()

# Plot for 'target' values by 'data_block_id' for different 'county' values
g = sns.FacetGrid(df, col="county", col_wrap=4, height=4, aspect=1)
g.map(sns.lineplot, "data_block_id", "target")
g.set_titles("county = {col_name}")
g.set_axis_labels("Data Block ID", "Target Value")
plt.subplots_adjust(top=0.9)
g.fig.suptitle("Target Values by Data Block ID for Different Counties")
plt.show()


In [ ]:
# Filtering the data to include only rows where 'is_consumption' is 1
filtered_data = enefit[enefit['is_consumption'] == 1]

# Grouping the data by 'prediction_unit_id' and 'date'
grouped_data = filtered_data.groupby(['prediction_unit_id', 'date'])['target'].sum().reset_index()

# Pivoting the data for plotting
pivot_data = grouped_data.pivot(index='date', columns='prediction_unit_id', values='target')

# Plotting
plt.figure(figsize=(15, 8))
sns.lineplot(data=pivot_data)
plt.title('Target Value Changes Over Time by Prediction Unit ID (is_consumption = 1)')
plt.xlabel('Date')
plt.ylabel('Target Value')
plt.legend(title='Prediction Unit ID', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
# Identifying the top 2 'prediction_unit_id's with the highest average 'target' values
top_prediction_unit_ids = filtered_data.groupby('prediction_unit_id')['target'].mean().nlargest(2).index

# Excluding the top 2 'prediction_unit_id's from the filtered data
excluded_data = filtered_data[~filtered_data['prediction_unit_id'].isin(top_prediction_unit_ids)]

# Grouping the excluded data by 'prediction_unit_id' and 'date'
grouped_excluded_data = excluded_data.groupby(['prediction_unit_id', 'date'])['target'].mean().reset_index()

# Pivoting the data for plotting
pivot_excluded_data = grouped_excluded_data.pivot(index='date', columns='prediction_unit_id', values='target')

# Plotting
plt.figure(figsize=(15, 8))
sns.lineplot(data=pivot_excluded_data)
plt.title('Target Value Changes Over Time (Excluding Top 2 Prediction Unit IDs)')
plt.xlabel('Date')
plt.ylabel('Target Value')
plt.legend(title='Prediction Unit ID', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


# Modeling

## Linear regression

In [ ]:
# Check the unique values in the 'county', 'product_type', and 'data_block_id' columns
unique_values = {
    "county": df['county'].unique(),
    "product_type": df['product_type'].unique()
    }

# Create dummy variables for 'county' and 'product_type'
df_dummies_county = pd.get_dummies(df['county'], prefix='county', drop_first=True)
df_dummies_product_type = pd.get_dummies(df['product_type'], prefix='product_type', drop_first=True)

# Drop the original categorical columns from the dataframe
df_dummy = df.drop(['county', 'product_type'], axis=1)

# Concatenate the dummy variables with the original dataframe
df_dummy = pd.concat([df, df_dummies_county, df_dummies_product_type], axis=1)

# Define the dependent variable and independent variables
y = df_dummy['target']
X = df_dummy.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

# Predicting on the filtered dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Performing a normality test on the residuals
normality_test_result = stats.shapiro(residuals)

# Calculating the model performance metrics
rmse = mean_squared_error(y, y_hat) ** .5
r2 = r2_score(y, y_hat)

print('rmse:', rmse)
print('r2:', r2)
print('normality_test_result:', normality_test_result)

# Plotting the residuals
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals, s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# Generating a Q-Q plot for the residuals
plt.figure(figsize=(15, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.show()


In [ ]:
# Define the counties to select for one-hot encoding
counties_to_select = ['county_0', 'county_2', 'county_11', 'county_13', 'county_14']

# Create dummy variables for 'county' including only the specified counties
df_dummies_county = pd.get_dummies(df['county'], prefix='county')
df_dummies_product_type = pd.get_dummies(df['product_type'], prefix='product_type', drop_first=True)

df_dummies_county = df_dummies_county[counties_to_select]

# Concatenate the dummy variables with the original dataframe
df_dummy_1 = pd.concat([df, df_dummies_county, df_dummies_product_type], axis=1)

# Drop the original 'county' and 'product_type' columns
df_dummy_1 = df_dummy_1.drop(['county', 'product_type'], axis=1)

# Define the dependent variable and independent variables
y = df_dummy_1['target']
X = df_dummy_1.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

# Predicting on the filtered dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Performing a normality test on the residuals
normality_test_result = stats.shapiro(residuals)

# Calculating the model performance metrics
rmse = mean_squared_error(y, y_hat) ** .5
r2 = r2_score(y, y_hat)

print('rmse:', rmse)
print('r2:', r2)
print('normality_test_result:', normality_test_result)

# Plotting the residuals
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals, s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# Generating a Q-Q plot for the residuals
plt.figure(figsize=(15, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.show()


In [ ]:
# Get the unique counties
unique_counties = df['county'].unique()

# Create a figure
plt.figure(figsize=(15, 6 * len(unique_counties)))

for i, county in enumerate(unique_counties):
    # Filter the dataframe for the current county
    df_county = df[df['county'] == county]

    # Create dummy variables for 'product_type'
    df_dummies_product_type = pd.get_dummies(df_county['product_type'], prefix='product_type', drop_first=True)

    # Drop the original categorical columns from the dataframe
    df_county = df_county.drop(['county', 'product_type'], axis=1)

    # Concatenate the dummy variables with the original dataframe
    df_county = pd.concat([df_county, df_dummies_product_type], axis=1)

    # Define the dependent variable and independent variables
    y = df_county['target']
    X = df_county.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

    # Adding a constant to the model (intercept)
    X = sm.add_constant(X)

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Predicting on the filtered dataset
    y_hat = model.predict(X)

    # Calculating residuals
    residuals = y - y_hat

    # Plotting the residuals
    plt.subplot(len(unique_counties), 1, i+1)
    plt.scatter(y.index, residuals, s=20)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.title(f'Residual Plot for County {county}')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')

plt.tight_layout()
plt.show()

In [ ]:
# Filter the dataframe to exclude 'county_11'
df_filtered = df[df['county'] != 'county_11']

# Create dummy variables for 'county' and 'product_type'
df_dummies_county = pd.get_dummies(df_filtered['county'], prefix='county', drop_first=True)
df_dummies_product_type = pd.get_dummies(df_filtered['product_type'], prefix='product_type', drop_first=True)

# Drop the original categorical columns from the dataframe
df_filtered = df_filtered.drop(['county', 'product_type'], axis=1)

# Concatenate the dummy variables with the original dataframe
df_filtered = pd.concat([df_filtered, df_dummies_county, df_dummies_product_type], axis=1)

# Define the dependent variable and independent variables
y = df_filtered['target']
X = df_filtered.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Predicting on the filtered dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Plotting the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y.index, residuals, s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot Excluding County_11')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

plt.tight_layout()


In [ ]:
# Filter the dataframe to include only the desired county values
df_filtered = df[df['county'].isin([0, 2, 11, 13, 14])]

# Calculate the mean target value for each 'data_block_id'
mean_target = df_filtered[df_filtered['county'] != 'other'].groupby('data_block_id')['target'].mean()

# Replace the target values in the 'other' county column with the mean values
df_filtered.loc[df_filtered['county'] == 'other', 'target'] = df_filtered.loc[df_filtered['county'] == 'other', 'data_block_id'].map(mean_target)

# Perform one-hot encoding on the 'county' and 'product_type' columns
df_new = pd.get_dummies(df_filtered, columns=['county', 'product_type'], drop_first=True)

# Define the dependent variable and independent variables
y = df_new['target']
X = df_new.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

# Predicting on the filtered dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Performing a normality test on the residuals
normality_test_result = stats.shapiro(residuals)

# Calculating the model performance metrics
rmse = mean_squared_error(y, y_hat) ** .5
r2 = r2_score(y, y_hat)

print('rmse:', rmse)
print('r2:', r2)
print('normality_test_result:', normality_test_result)

# Plotting the residuals
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals, s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# Generating a Q-Q plot for the residuals
plt.figure(figsize=(15, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.show()


In [ ]:
# Filter the dataframe to include only the desired county values
df_filtered = df[df['county'].isin([0, 2, 11, 13, 14])]

# Perform one-hot encoding on the 'county' and 'product_type' columns
df_new = pd.get_dummies(df_filtered, columns=['county', 'product_type'], drop_first=True)

# Define the dependent variable and independent variables
y = df_new['target']
X = df_new.drop(['target'], axis=1)  # Dropping 'Unnamed: 0' as it seems like an index

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

# Predicting on the filtered dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Performing a normality test on the residuals
normality_test_result = stats.shapiro(residuals)

# Calculating the model performance metrics
rmse = mean_squared_error(y, y_hat) ** .5
r2 = r2_score(y, y_hat)

print('rmse:', rmse)
print('r2:', r2)
print('normality_test_result:', normality_test_result)

# Plotting the residuals
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals, s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

# Generating a Q-Q plot for the residuals
plt.figure(figsize=(15, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.show()

## GMM

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

# Assuming df_dummy is your one-hot encoded DataFrame
y = df_dummy['target']
X = df_dummy.drop(['target'], axis=1)

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Predicting on the dataset
y_hat = model.predict(X)

# Calculating residuals
residuals = y - y_hat

# Reshape the residuals to 2D array as required by the GaussianMixture model
residuals_2d = residuals.values.reshape(-1, 1)

# Fit a Gaussian Mixture Model with 3 components on the residuals
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(residuals_2d)

# Predict the labels for the data points
gmm_labels = gmm.predict(residuals_2d)

# Convert the GMM labels to one-hot encoded features
gmm_features = pd.get_dummies(gmm_labels, prefix='gmm', drop_first=True)

# Add the GMM features to the dataset
df_dummy = pd.concat([df_dummy, gmm_features], axis=1)

# Plotting the residuals with different colors for each Gaussian component
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals, c=gmm_labels, cmap='viridis', s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot with Different Colors for Gaussian Components')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.colorbar(label='Gaussian Component')
plt.show()



# Define the dependent variable and independent variables
y = df_dummy['target']
X = df_dummy.drop(['target'], axis=1)

# Adding a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model with the GMM feature
model_with_gmm = sm.OLS(y, X).fit()

# Predicting on the dataset with the GMM feature
y_hat_with_gmm = model_with_gmm.predict(X)

# Calculating residuals with the GMM feature
residuals_with_gmm = y - y_hat_with_gmm

# Calculate the performance metrics with the GMM feature
rmse_with_gmm = mean_squared_error(y, y_hat_with_gmm) ** .5
r2_with_gmm = r2_score(y, y_hat_with_gmm)

# Print the performance metrics
print("Performance metrics with GMM feature:")
print("RMSE:", rmse_with_gmm)
print("R-squared:", r2_with_gmm)

# Plotting the residuals with different colors for each Gaussian component
plt.figure(figsize=(15, 6))
plt.scatter(y.index, residuals_with_gmm, c=gmm_labels, cmap='viridis', s=20)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot with GMM Included')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.colorbar(label='Gaussian Component')
plt.show()
